# parsing хайтек

In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup as bs
import re
from tqdm.notebook import trange, tqdm
from tqdm.gui import tqdm as tqdm_gui
from IPython.display import display

импорт библиотек

In [2]:
pd.set_option('display.max_colwidth', 50)
pd.set_option('display.max_columns', 1000)

In [3]:
reqx = requests.get('https://hi-tech.md/kompyuternaya-tehnika/sport-i-otdyh/hd-cvi-kamery/ip-kamery/')

In [4]:
soup = bs(reqx.text, 'html.parser')

In [5]:
data = pd.DataFrame({'soup_name':pd.Series(soup.findAll('div', class_='ypi-grid-list__item_body'),dtype = 'object')})
data.head()

,soup_name
0,"[[ , [\n, <input name=""appearance[show_sku]"" t..."
1,"[[ , [\n, <input name=""appearance[show_sku]"" t..."
2,"[[ , [\n, <input name=""appearance[show_sku]"" t..."
3,"[[ , [\n, <input name=""appearance[show_sku]"" t..."
4,"[[ , [\n, <input name=""appearance[show_sku]"" t..."


парсинг тестовой страници сайта 

In [6]:
def names_correct(x):
    name = str(x)
    return  name 

In [7]:
data['soup_name'] = data['soup_name'].apply(names_correct)

меняем данные парсинга на текстовый тип 

In [8]:
def find_id(x):
    find = x.find('span class="ty-control-group__item"')
    start = find+len('span class="ty-control-group__item"')+1
    stop = find+12+ len('span class="ty-control-group__item"')
    xxx = x[start: stop]
    return xxx

In [9]:
data['id'] = data['soup_name'].apply(find_id)

находим в тексте и созаем столбец с кодами товара

In [10]:
data['date'] = pd.to_datetime('today').normalize()

создаем столбец с датой парсинга

In [11]:
def find_availability(t):
    find = t.find('i class="material-icons"')
    find = t.find('</i>', find)
    start = find+len('</i>')
    stop = t.find('\n', find)
    xxx = t[start: stop]
    return xxx

In [12]:
data['availability'] = data['soup_name'].apply(find_availability)

находим в тексте и создаем столбец с наличием товара

In [13]:
def find_title(t):
    find = t.find('a class="product-title"')
    find = t.find('title=', find)
    start = find+len('title=')+1
    stop = t.find('>', find)-1
    xxx = t[start: stop]
    return xxx


In [14]:
data['title'] = data['soup_name'].apply(find_title)

находим в тексте и создаем столбец снаименованием товара

In [15]:
def find_price(t):
    find = t.find('span class="ty-price-num"')
    find = t.find('>', find)
    start = find+len('>')
    stop = t.find('<', find)
    xxx = t[start: stop]
    xxx = xxx.replace(u'\xa0',u'')
    return xxx

In [16]:
data['price'] = data['soup_name'].apply(find_price)

Находим в тексте и создаем столбец с ценой товара

In [17]:
def find_old_price(t):
    find = t.find('span class="ty-strike"')
    find = t.find('span class="ty-list-price ty-nowrap"',find)
    find = t.find('>', find)
    start = find+len('>')
    stop = t.find('<', find)
    xxx = t[start: stop]
    xxx = xxx.replace(u'\xa0',u'')
    return xxx

In [18]:
data['old_price'] = data['soup_name'].apply(find_old_price)

Для акционного товара Находим в тексте и создаем столбец со старой ценой до начала акции

In [19]:
def find_url(t):
    find = t.find('a class="product-title"')
    find = t.find('href="', find)
    start = find+len('href="')
    stop = t.find('title', find)-2
    xxx = t[start: stop]
    return xxx

In [20]:
data['soup_name'] = data['soup_name'].apply(find_url)

Находим в тексте информацию об адресе совара и переписываем эту информацию вместо парсинговой.

In [21]:
def preprocessing(x):
    print(f'таблица имеет следующий вид:')
    display(x.head())
    print('*'*100)
    print(f'таблица имеет {x.shape[0]} сторок и {x.shape[1]} столбцов')
    print('*'*100)
    print(f'в таблице обнаружено дублекатов:{x.duplicated().sum()}')
    print('*'*100)
    print('в таблице обнаружены пропуски в следующих столбцах:')
    display(x.isna().sum())
    print('*'*100)
    print('Эти пропуски составлябт следующее количество в процентах')
    display((x.isna().mean()* 100).round(1))
    print('*'*100)
    print('столбцы имеют следующие типы:')
    display(x.dtypes)
    x.columns = [y.lower().replace(' ', '_') for y in x.columns.values] 
    print('*'*100)
    print('названия столбцов переписаны в нижнем регистре, пробелы заменены на нижнее подчеркивание')
    print('*'*100)
    display(x.info())

создаем функцию предварительного просмотра данных

In [22]:
def parsing_2(url):
    data = pd.DataFrame(columns=['soup_name', 'id', 'date', 'availability', 'title', 'price', 'old_price'])
    for i in tqdm(url):        
        q = 1
        while q>0:        
            reqx = requests.get(i + str(q))
            soup = bs(reqx.text, 'html.parser')
            datax = pd.DataFrame({'soup_name':pd.Series(soup.findAll('div', class_='ypi-grid-list__item_body'),dtype = 'object')})
            data = pd.concat([data, datax], ignore_index=True)
            if len(list(soup.findAll('div', class_='ypi-grid-list__item_body'))) > 0:
                q = q + 1
            else:
                q = 0
    data['soup_name'] = data['soup_name'].apply(names_correct)
    data['id'] = data['soup_name'].apply(find_id)
    data['date'] = pd.to_datetime('today').normalize()
    data['availability'] = data['soup_name'].apply(find_availability)
    data['title'] = data['soup_name'].apply(find_title)
    data['price'] = data['soup_name'].apply(find_price)
    data['old_price'] = data['soup_name'].apply(find_old_price)
    data['soup_name'] = data['soup_name'].apply(find_url)
    data['price'] = data['price'].astype(float)
    data.loc[data['old_price'] == '','old_price'] = np.nan
    data['old_price'] = data['old_price'].astype(float)
    data = data.drop_duplicates()
    return data

создаем функцию, которая по списку url проходит и парсит всю доступную информацию, преобразуя ее в датафрейм.

all_url = ['https://hi-tech.md/televizory-i-elektronika/page-',
       'https://hi-tech.md/bytovaya-tehnika/page-',
       'https://hi-tech.md/kompyuternaya-tehnika/page-']

all_url = ['https://hi-tech.md/bytovaya-tehnika/page-']

In [23]:
all_url = ['https://hi-tech.md/televizory-i-elektronika/page-',
           'https://hi-tech.md/bytovaya-tehnika/page-',
           'https://hi-tech.md/kompyuternaya-tehnika/page-',
           'https://hi-tech.md/mebel-tekstil/kompyuternaya-mebel/ofisnye-kresla-i-stulya/komp.-kresla/page-',
           'https://hi-tech.md/instrumenty-i-oborudovanie/elektroinstrument/page-',
           'https://hi-tech.md/tovary-dlya-doma/page-'
          ]

список url по которым будет парсится сайт

In [24]:
df = parsing_2(all_url)
df

  0%|          | 0/6 [00:00<?, ?it/s]

,soup_name,id,date,availability,title,price,old_price
0,https://hi-tech.md/televizory-i-elektronika/ba...,Т-000088228,2024-06-09,В наличии,Батарейка Energenie EG-BA-CR1220-01,4.0,NaN
1,https://hi-tech.md/televizory-i-elektronika/ba...,Т-000088229,2024-06-09,В наличии,Батарейка Energenie EG-BA-CR2016-01,4.0,NaN
2,https://hi-tech.md/televizory-i-elektronika/ba...,Т-000088221,2024-06-09,В наличии,Батарейка Energenie EG-BA-AA4-01,5.0,NaN
3,https://hi-tech.md/televizory-i-elektronika/ba...,Т-000088222,2024-06-09,В наличии,Батарейка Energenie EG-BA-AAA4-01,5.0,NaN
4,https://hi-tech.md/televizory-i-elektronika/ba...,Т-000088230,2024-06-09,В наличии,Батарейка Energenie EG-BA-CR2025-01,5.0,NaN
...,...,...,...,...,...,...,...
9288,https://hi-tech.md/tovary-dlya-doma/osvescheni...,Т-000085036,2024-06-09,В наличии,Люстра 4PL SATIPO TRANSPARENT LED15W E27 (5628...,3279.0,4099.0
9289,https://hi-tech.md/tovary-dlya-doma/osvescheni...,Т-000085037,2024-06-09,В наличии,Люстра 4PL SATIPO AMBER LED15W E27 (5625) (TKL),3311.0,4139.0
9290,https://hi-tech.md/tovary-dlya-doma/osvescheni...,Т-000085021,2024-06-09,В наличии,Люстра 3PL UNIVERSO BLACK LED15W E27 (4483) (TKL),3359.0,4199.0
9291,https://hi-tech.md/tovary-dlya-doma/osvescheni...,Т-000085033,2024-06-09,В наличии,Люстра 6PL FAIRY BLACK 6W G9 (4856) (TKL),3471.0,4339.0


полученная информация

In [25]:
preprocessing(df)

таблица имеет следующий вид:


,soup_name,id,date,availability,title,price,old_price
0,https://hi-tech.md/televizory-i-elektronika/ba...,Т-000088228,2024-06-09,В наличии,Батарейка Energenie EG-BA-CR1220-01,4.0,NaN
1,https://hi-tech.md/televizory-i-elektronika/ba...,Т-000088229,2024-06-09,В наличии,Батарейка Energenie EG-BA-CR2016-01,4.0,NaN
2,https://hi-tech.md/televizory-i-elektronika/ba...,Т-000088221,2024-06-09,В наличии,Батарейка Energenie EG-BA-AA4-01,5.0,NaN
3,https://hi-tech.md/televizory-i-elektronika/ba...,Т-000088222,2024-06-09,В наличии,Батарейка Energenie EG-BA-AAA4-01,5.0,NaN
4,https://hi-tech.md/televizory-i-elektronika/ba...,Т-000088230,2024-06-09,В наличии,Батарейка Energenie EG-BA-CR2025-01,5.0,NaN


****************************************************************************************************
таблица имеет 8722 сторок и 7 столбцов
****************************************************************************************************
в таблице обнаружено дублекатов:0
****************************************************************************************************
в таблице обнаружены пропуски в следующих столбцах:


soup_name          0
id                 0
date               0
availability       0
title              0
price              0
old_price       6584
dtype: int64

****************************************************************************************************
Эти пропуски составлябт следующее количество в процентах


soup_name        0.0
id               0.0
date             0.0
availability     0.0
title            0.0
price            0.0
old_price       75.5
dtype: float64

****************************************************************************************************
столбцы имеют следующие типы:


soup_name               object
id                      object
date            datetime64[ns]
availability            object
title                   object
price                  float64
old_price              float64
dtype: object

****************************************************************************************************
названия столбцов переписаны в нижнем регистре, пробелы заменены на нижнее подчеркивание
****************************************************************************************************
<class 'pandas.core.frame.DataFrame'>
Int64Index: 8722 entries, 0 to 9292
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   soup_name     8722 non-null   object        
 1   id            8722 non-null   object        
 2   date          8722 non-null   datetime64[ns]
 3   availability  8722 non-null   object        
 4   title         8722 non-null   object        
 5   price         8722 non-null   float64       
 6   old_price     2138 non-null   float64       
dtypes: datetime64[ns](1), float64(2), object(4)
memory usage: 545.1+ KB


None

In [26]:
today = pd.to_datetime('today').normalize()
today = today.to_pydatetime()
df.to_csv('csv/hi_tech_'+str(today.year)+'_'+str(today.month)+'_'+str(today.day)+'.csv', index=False)  

Переписывем датафрейм в файл

# аналитика хайтек

In [27]:
brend = pd.read_excel('csv/brend.xlsx')

In [28]:
def lower(x):
    return x.lower()

In [29]:
brend['names'] = brend['name'].apply(lower)

In [30]:
brend_up = brend.name.unique()
brend_low = brend.names.unique()

загружаем и создаем переменную с брендами

In [31]:
df

,soup_name,id,date,availability,title,price,old_price
0,https://hi-tech.md/televizory-i-elektronika/ba...,Т-000088228,2024-06-09,В наличии,Батарейка Energenie EG-BA-CR1220-01,4.0,NaN
1,https://hi-tech.md/televizory-i-elektronika/ba...,Т-000088229,2024-06-09,В наличии,Батарейка Energenie EG-BA-CR2016-01,4.0,NaN
2,https://hi-tech.md/televizory-i-elektronika/ba...,Т-000088221,2024-06-09,В наличии,Батарейка Energenie EG-BA-AA4-01,5.0,NaN
3,https://hi-tech.md/televizory-i-elektronika/ba...,Т-000088222,2024-06-09,В наличии,Батарейка Energenie EG-BA-AAA4-01,5.0,NaN
4,https://hi-tech.md/televizory-i-elektronika/ba...,Т-000088230,2024-06-09,В наличии,Батарейка Energenie EG-BA-CR2025-01,5.0,NaN
...,...,...,...,...,...,...,...
9288,https://hi-tech.md/tovary-dlya-doma/osvescheni...,Т-000085036,2024-06-09,В наличии,Люстра 4PL SATIPO TRANSPARENT LED15W E27 (5628...,3279.0,4099.0
9289,https://hi-tech.md/tovary-dlya-doma/osvescheni...,Т-000085037,2024-06-09,В наличии,Люстра 4PL SATIPO AMBER LED15W E27 (5625) (TKL),3311.0,4139.0
9290,https://hi-tech.md/tovary-dlya-doma/osvescheni...,Т-000085021,2024-06-09,В наличии,Люстра 3PL UNIVERSO BLACK LED15W E27 (4483) (TKL),3359.0,4199.0
9291,https://hi-tech.md/tovary-dlya-doma/osvescheni...,Т-000085033,2024-06-09,В наличии,Люстра 6PL FAIRY BLACK 6W G9 (4856) (TKL),3471.0,4339.0


In [32]:
def category(x):
    xx = x.split('/')
    xx = xx[3].replace('-','_')
    return xx    

In [33]:
def subcategory(x):
    xx = x.split('/')
    xx = xx[4].replace('-','_')
    return xx   

In [34]:
def subgroup(x):
    xx = x.split('/')
    xx = xx[-3].replace('-','_')
    return xx   

In [35]:
def group(x):
    xx = x.split('/')
    if len(xx) == 7:
        return xx[-3].replace('-','_')
    elif len(xx) == 8:
        return xx[-3].replace('-','_')
    elif len(xx) == 9:
        return xx[-4].replace('-','_')
    elif len(xx) == 10:
        return xx[-5].replace('-','_')
    elif len(xx) == 11:
        return xx[-6].replace('-','_')  

создаем вспомогательные функции

In [36]:
df['action_sale'] = 0

In [37]:
df.loc[df['old_price'] > 0,'action_sale'] = 1

создаем столбец который показывает является ли товар акционным(1 - акционный, 0 - не акционный)

In [38]:
df['sale'] = (100 - ((df['price']*100)/df['old_price'])).round(2) 

создаем столбец показывающий скидку в процентах

In [39]:
df['category'] = df['soup_name'].apply(category)

Создаем столбец показывающий категорию товаров

In [40]:
df['subcategory'] = df['soup_name'].apply(subcategory)

создаем столбец показывающий подкатегорию товаров

In [41]:
df['group'] = df['soup_name'].apply(group)

создаем столбец показывающий группу товаров

In [42]:
df.loc[df['subcategory']=='monitory','group'] = 'monitory'

в подкатегории мониторы меняем названия групп на мониторы

In [43]:
df.head()

,soup_name,id,date,availability,title,price,old_price,action_sale,sale,category,subcategory,group
0,https://hi-tech.md/televizory-i-elektronika/ba...,Т-000088228,2024-06-09,В наличии,Батарейка Energenie EG-BA-CR1220-01,4.0,NaN,0,NaN,televizory_i_elektronika,batareyki_i_zaryadnye_ustroystva,batareyki_i_zaryadnye_ustroystva
1,https://hi-tech.md/televizory-i-elektronika/ba...,Т-000088229,2024-06-09,В наличии,Батарейка Energenie EG-BA-CR2016-01,4.0,NaN,0,NaN,televizory_i_elektronika,batareyki_i_zaryadnye_ustroystva,batareyki_i_zaryadnye_ustroystva
2,https://hi-tech.md/televizory-i-elektronika/ba...,Т-000088221,2024-06-09,В наличии,Батарейка Energenie EG-BA-AA4-01,5.0,NaN,0,NaN,televizory_i_elektronika,batareyki_i_zaryadnye_ustroystva,batareyki_i_zaryadnye_ustroystva
3,https://hi-tech.md/televizory-i-elektronika/ba...,Т-000088222,2024-06-09,В наличии,Батарейка Energenie EG-BA-AAA4-01,5.0,NaN,0,NaN,televizory_i_elektronika,batareyki_i_zaryadnye_ustroystva,batareyki_i_zaryadnye_ustroystva
4,https://hi-tech.md/televizory-i-elektronika/ba...,Т-000088230,2024-06-09,В наличии,Батарейка Energenie EG-BA-CR2025-01,5.0,NaN,0,NaN,televizory_i_elektronika,batareyki_i_zaryadnye_ustroystva,batareyki_i_zaryadnye_ustroystva


In [44]:
df['category'].unique()

array(['televizory_i_elektronika', 'kompyuternaya_tehnika',
       'diskont_ne_deystvuet_ru', 'bytovaya_tehnika', 'tovar_po_akcii',
       'instrumenty_i_oborudovanie', 'mebel_tekstil', 'akciya',
       'tovary_dlya_doma'], dtype=object)

In [45]:
df['subcategory'].unique()

array(['batareyki_i_zaryadnye_ustroystva', 'nastennye_krepleniya_dlya_tv',
       'audiotehnika', 'smartfony_i_aksessuary', 'multimedia',
       'pristavki_dlya_tv', 'gadzhety', 'telefony_dect_i_provodnye',
       'tovary_apple', 'sport_i_otdyh',
       'hd_cvi_videokamera_dahua_dh_hac_hdw1220mp_0280b_s2',
       'umnyy_dom_s_yandeks', 'televizory', 'pylesosy_i_aksessuary',
       'tehnika_karcher', 'aksessuary_dlya_melkoy_bytovoy_tehniki',
       'tovary_dlya_uhoda_za_odezhdoy', 'minimoyki',
       'prigotovlenie_napitkov', 'melkaya_tehnika_dlya_kuhni',
       'tehnika_dlya_krasoty', 'stiralnye_i_sushilnye_mashiny',
       'krupnaya_tehnika_dlya_kuhni', 'tehnika_dlya_zdorovya',
       'klimaticheskaya_tehnika', 'ustanovka_bytovoy_tehniki',
       'prigotovlenie_pischi', 'teploventilyator_tesy_hl_213v',
       'elektroplita_zilan_zln8092', 'tekhnika_dlya_doma',
       'kolpachok_rj_45_gembird_bt5bk_5', 'nakopiteli',
       'konnektor_rj_45_gembird_cat5e_lc_8p8c_001',
       'setevoe_ob

In [46]:
df.group.unique()

array(['batareyki_i_zaryadnye_ustroystva', 'nastennye_krepleniya_dlya_tv',
       'mikrofony', 'naushniki', 'aksessuary', 'smartfony_i_aksessuary',
       'akusticheskie_sistemy', 'akustika', 'pristavki_dlya_tv',
       'muzykalnaya_tehnika', 'pleery', 'umny_dom', 'radio',
       'telefony_dect_i_provodnye', 'saundbary', 'airpods',
       'smart_gadzhety', 'hd_cvi_kamery', None, 'yandeks_stancii',
       'televizory', 'smartfony', 'homepod', 'apple_whatch', 'iphone',
       'filtry_i_meshki_dlya_pyli', 'meshki_i_filtry_karcher',
       'aksessuary_k_pylesosam', 'aksessuary_dlya_zubnyh_schetok',
       'mashinki_dlya_udaleniya_katyshkov', 'minimoyki',
       'gladilnye_doski', 'sushilki_dlya_belya', 'turki_elektricheskie',
       'vakuumatory', 'vypryamiteli_dlya_volos',
       'frontalnye_stiralnye_mashiny', 'kofevarki', 'vesy_napolnye',
       'kofe', 'miksery', 'vesy_kuhonnye', 'vytyazhki', 'feny_dlya_volos',
       'elektricheskie_chayniki', 'varochnye_poverhnosti', 'ployki',
      

проссматриваем названия категорий подкатегорий и групп товаров

In [47]:
df.loc[df['group'].isna(),'group'] = 'No_group'

удалеем из столбца group все значения None

In [48]:
def count_group(x):
    return df[df['group']==x]['id'].count()    

In [49]:
tqdm.pandas()
df['count_group'] = df['group'].progress_apply(count_group)

  0%|          | 0/8722 [00:00<?, ?it/s]

создание столбца с количеством товара в группе

In [50]:
def count_group_presence(x):
    return df[(df['availability']=='В наличии')&(df['group']==x)]['id'].count()   

In [51]:
tqdm.pandas()
df['count_group_presence'] = df['group'].progress_apply(count_group_presence)

  0%|          | 0/8722 [00:00<?, ?it/s]

создание столбца с количеством товара, который есть в наличии, в группе

In [52]:
def dif_price(x):    
    max_price = df[(df['availability']=='В наличии')&(df['group']==x)]['price'].max()
    min_price = df[(df['availability']=='В наличии')&(df['group']==x)]['price'].min() 
    return max_price / min_price

In [53]:
tqdm.pandas()
df['dif_price'] = df['group'].progress_apply(dif_price)

  0%|          | 0/8722 [00:00<?, ?it/s]

создам столбец с отношением самой дорогой цены в группе к самой дешевой.

In [54]:
def brends(x):
    xx = x.replace('\xa0', ' ')
    xx = xx.split(' ')    
    for i in range(len(xx)-1):
        if xx[i].lower() in brend_low:
            return xx[i].lower()
        elif (str(xx[i].lower())+' '+str(xx[i+1].lower())) in brend_low:
            return (str(xx[i].lower())+' '+str(xx[i+1].lower()))    
    else:    
        return 'другой'

In [55]:
tqdm.pandas()
df['brend'] = df['title'].progress_apply(brends)

  0%|          | 0/8722 [00:00<?, ?it/s]

In [56]:
df['group_prise'] = df.groupby('group')['price'].transform(lambda x: pd.cut(x, bins = 4, labels=["бюджет", "стандарт", "элит", "топ"]))

In [57]:
today = pd.to_datetime('today').normalize()
today = today.to_pydatetime()
df.to_csv('csv/hi_tech_sity_'+str(today.year)+'_'+str(today.month)+'_'+str(today.day)+'.csv', index=False)  

# Добавление остатков по магазинам

In [58]:
def count_sity_2(x):
    x['reqx'] = requests.get(x['soup_name'])
    x['soup'] = bs(reqx.text, 'html.parser')
    x['element'] = soup.findAll('div', class_='ty-warehouses-store__availability')
    x['e1'] = (x['element'][0].text.strip().replace('В наличии', '1').replace('Нет в наличии','0')
    .replace('Под заказ (1-3 дня)','0').replace('nan','0'))
    x['e2'] = (x['element'][1].text.strip().replace('В наличии', '1').replace('Нет в наличии','0')
    .replace('Под заказ (1-3 дня)','0').replace('nan','0'))
    return  x['e1'], x['e2']
    
    

In [59]:
def count_sity(x):
    reqx = requests.get(x)
    soup = bs(reqx.text, 'html.parser')    
    element = soup.findAll('div', class_='ty-warehouses-store__availability')
    data = []
    if len(element) == 14:
        data.append(element[0].text.strip().replace('В наличии', '1').replace('Нет в наличии','0')
        .replace('Под заказ (1-3 дня)','0').replace('nan','0'))    
        data.append(element[1].text.strip().replace('В наличии', '1').replace('Нет в наличии','0')
        .replace('Под заказ (1-3 дня)','0').replace('nan','0'))
        data.append(element[2].text.strip().replace('В наличии', '1').replace('Нет в наличии','0')
        .replace('Под заказ (1-3 дня)','0').replace('nan','0'))
        data.append(element[3].text.strip().replace('В наличии', '1').replace('Нет в наличии','0')
        .replace('Под заказ (1-3 дня)','0').replace('nan','0'))
        data.append(element[4].text.strip().replace('В наличии', '1').replace('Нет в наличии','0')
        .replace('Под заказ (1-3 дня)','0').replace('nan','0')) 
        data.append(element[5].text.strip().replace('В наличии', '1').replace('Нет в наличии','0')
        .replace('Под заказ (1-3 дня)','0').replace('nan','0'))
        data.append(element[6].text.strip().replace('В наличии', '1').replace('Нет в наличии','0')
        .replace('Под заказ (1-3 дня)','0').replace('nan','0'))
        data.append(element[7].text.strip().replace('В наличии', '1').replace('Нет в наличии','0')
        .replace('Под заказ (1-3 дня)','0').replace('nan','0'))
        data.append(element[8].text.strip().replace('В наличии', '1').replace('Нет в наличии','0')
        .replace('Под заказ (1-3 дня)','0').replace('nan','0'))
        data.append(element[9].text.strip().replace('В наличии', '1').replace('Нет в наличии','0')
        .replace('Под заказ (1-3 дня)','0').replace('nan','0'))
        data.append(element[10].text.strip().replace('В наличии', '1').replace('Нет в наличии','0')
        .replace('Под заказ (1-3 дня)','0').replace('nan','0'))
        data.append(element[11].text.strip().replace('В наличии', '1').replace('Нет в наличии','0')
        .replace('Под заказ (1-3 дня)','0').replace('nan','0'))
        data.append(element[12].text.strip().replace('В наличии', '1').replace('Нет в наличии','0')
        .replace('Под заказ (1-3 дня)','0').replace('nan','0'))
        data.append(element[13].text.strip().replace('В наличии', '1').replace('Нет в наличии','0')
        .replace('Под заказ (1-3 дня)','0').replace('nan','0'))
        data.append(0)        
        return data
    else:
        data.append(0)
        data.append(0)
        data.append(0)
        data.append(0)
        data.append(0)
        data.append(0)
        data.append(0)
        data.append(0)
        data.append(0)
        data.append(0)
        data.append(0)
        data.append(0)
        data.append(0)
        data.append(0)
        data.append(1)
        return data


In [60]:
tqdm.pandas()
df['count_sity']= df['soup_name'].progress_apply(count_sity)

  0%|          | 0/8722 [00:00<?, ?it/s]

t = pd.DataFrame({ 'id': df['id'] ,
                  'Тир_Юности_14А': df['count_sity'].str[0], 
                  'Ц_Тир_25_Октября_69': df['count_sity'].str[1],
                  'Ц_Бенд_Спортивная_5': df['count_sity'].str[2],
                  'Бенд_Советская_5': df['count_sity'].str[3],
                  'Бенд_Советская_8': df['count_sity'].str[4], 
                  'Слоб_Ленина_97А': df['count_sity'].str[5],
                  'Днестр_Строителей_18': df['count_sity'].str[6],
                  'Первом_Ленина_88': df['count_sity'].str[7],
                  'Григ_КМаркса_181А': df['count_sity'].str[8], 
                  'Григ_КМаркса_172': df['count_sity'].str[9],
                  'Дуб_Горького_29': df['count_sity'].str[10], 
                  'Рыбн_Вальченко_43/1': df['count_sity'].str[11], 
                  'Рыбн_Кирова_135': df['count_sity'].str[12],
                  'Кам_Кирова_240': df['count_sity'].str[13]})
t

df = df.merge(t, on='id')

In [61]:
df

,soup_name,id,date,availability,title,price,old_price,action_sale,sale,category,subcategory,group,count_group,count_group_presence,dif_price,brend,group_prise,count_sity
0,https://hi-tech.md/televizory-i-elektronika/ba...,Т-000088228,2024-06-09,В наличии,Батарейка Energenie EG-BA-CR1220-01,4.0,NaN,0,NaN,televizory_i_elektronika,batareyki_i_zaryadnye_ustroystva,batareyki_i_zaryadnye_ustroystva,76,76,199.750000,energenie,бюджет,"[1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0]"
1,https://hi-tech.md/televizory-i-elektronika/ba...,Т-000088229,2024-06-09,В наличии,Батарейка Energenie EG-BA-CR2016-01,4.0,NaN,0,NaN,televizory_i_elektronika,batareyki_i_zaryadnye_ustroystva,batareyki_i_zaryadnye_ustroystva,76,76,199.750000,energenie,бюджет,"[1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0]"
2,https://hi-tech.md/televizory-i-elektronika/ba...,Т-000088221,2024-06-09,В наличии,Батарейка Energenie EG-BA-AA4-01,5.0,NaN,0,NaN,televizory_i_elektronika,batareyki_i_zaryadnye_ustroystva,batareyki_i_zaryadnye_ustroystva,76,76,199.750000,energenie,бюджет,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0]"
3,https://hi-tech.md/televizory-i-elektronika/ba...,Т-000088222,2024-06-09,В наличии,Батарейка Energenie EG-BA-AAA4-01,5.0,NaN,0,NaN,televizory_i_elektronika,batareyki_i_zaryadnye_ustroystva,batareyki_i_zaryadnye_ustroystva,76,76,199.750000,energenie,бюджет,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0]"
4,https://hi-tech.md/televizory-i-elektronika/ba...,Т-000088230,2024-06-09,В наличии,Батарейка Energenie EG-BA-CR2025-01,5.0,NaN,0,NaN,televizory_i_elektronika,batareyki_i_zaryadnye_ustroystva,batareyki_i_zaryadnye_ustroystva,76,76,199.750000,energenie,бюджет,"[1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9288,https://hi-tech.md/tovary-dlya-doma/osvescheni...,Т-000085036,2024-06-09,В наличии,Люстра 4PL SATIPO TRANSPARENT LED15W E27 (5628...,3279.0,4099.0,1,20.0,tovary_dlya_doma,osveschenie,lyustry,194,173,41.589474,другой,топ,"[0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]"
9289,https://hi-tech.md/tovary-dlya-doma/osvescheni...,Т-000085037,2024-06-09,В наличии,Люстра 4PL SATIPO AMBER LED15W E27 (5625) (TKL),3311.0,4139.0,1,20.0,tovary_dlya_doma,osveschenie,lyustry,194,173,41.589474,другой,топ,"[0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]"
9290,https://hi-tech.md/tovary-dlya-doma/osvescheni...,Т-000085021,2024-06-09,В наличии,Люстра 3PL UNIVERSO BLACK LED15W E27 (4483) (TKL),3359.0,4199.0,1,20.0,tovary_dlya_doma,osveschenie,lyustry,194,173,41.589474,другой,топ,"[0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]"
9291,https://hi-tech.md/tovary-dlya-doma/osvescheni...,Т-000085033,2024-06-09,В наличии,Люстра 6PL FAIRY BLACK 6W G9 (4856) (TKL),3471.0,4339.0,1,20.0,tovary_dlya_doma,osveschenie,lyustry,194,173,41.589474,другой,топ,"[0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"


In [62]:
df['Тир_Юности_14А'] = df['count_sity'].str[0].astype('int')
df['Ц_Тир_25_Октября_69'] = df['count_sity'].str[1].astype('int')
df['ТИРАСПОЛЬ'] = df['Тир_Юности_14А'] + df['Ц_Тир_25_Октября_69']
df['Бенд_Спортивная_5'] = df['count_sity'].str[2].astype('int')
df['Бенд_Советская_5'] = df['count_sity'].str[3].astype('int')
df['Бенд_Советская_8'] = df['count_sity'].str[4].astype('int')
df['БЕНДЕРЫ'] = df['Бенд_Спортивная_5'] + df['Бенд_Советская_5'] + df['Бенд_Советская_8']
df['СЛОБОДЗЕЯ'] = df['count_sity'].str[5].astype('int')
df['ДНЕСТРОВСК'] = df['count_sity'].str[6].astype('int')
df['Первомайск'] = df['count_sity'].str[7].astype('int')
df['Григ_КМаркса_181А'] = df['count_sity'].str[8].astype('int')
df['Григ_КМаркса_172'] = df['count_sity'].str[9].astype('int')
df['Григориополь'] = df['Григ_КМаркса_181А'] + df['Григ_КМаркса_172']
df['ДУБОССАРЫ'] = df['count_sity'].str[10].astype('int')
df['Рыбн_Вальченко_43'] = df['count_sity'].str[11].astype('int')
df['Рыбн_Кирова_135'] = df['count_sity'].str[12].astype('int')
df['РЫБНИЦА'] = df['Рыбн_Вальченко_43'] + df['Рыбн_Кирова_135']
df['Каменка'] = df['count_sity'].str[13].astype('int')
df['error'] = df['count_sity'].str[14].astype('int')

In [70]:
df[df['error']==1].head(45)

,soup_name,id,date,availability,title,price,old_price,action_sale,sale,category,subcategory,group,count_group,count_group_presence,dif_price,brend,group_prise,тир_юности_14а,ц_тир_25_октября_69,тирасполь,бенд_спортивная_5,бенд_советская_5,бенд_советская_8,бендеры,слободзея,днестровск,первомайск,григ_кмаркса_181а,григ_кмаркса_172,григориополь,дубоссары,рыбн_вальченко_43,рыбн_кирова_135,рыбница,каменка,error
3165,https://hi-tech.md/bytovaya-tehnika/klimatiche...,T-900000003,2024-06-09,Предзаказ,Печь отопительно-варочная Огонь-батарея 7 антр...,5659.0,NaN,0,NaN,bytovaya_tehnika,klimaticheskaya_tehnika,otopitelnye_pechi,11,0,NaN,другой,бюджет,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3166,https://hi-tech.md/bytovaya-tehnika/klimatiche...,T-900000004,2024-06-09,Предзаказ,Печь отопительно-варочная Огонь-батарея 7 шоколад,5659.0,NaN,0,NaN,bytovaya_tehnika,klimaticheskaya_tehnika,otopitelnye_pechi,11,0,NaN,другой,бюджет,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3178,https://hi-tech.md/bytovaya-tehnika/klimatiche...,T-900000009,2024-06-09,Предзаказ,Печь отопительная Валериан 8 кВт антрацит EU,5769.0,NaN,0,NaN,bytovaya_tehnika,klimaticheskaya_tehnika,otopitelnye_pechi,11,0,NaN,квт,бюджет,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3234,https://hi-tech.md/bytovaya-tehnika/klimatiche...,T-900000005,2024-06-09,Предзаказ,Печь отопительно-варочная Огонь-батарея 9 антр...,6289.0,NaN,0,NaN,bytovaya_tehnika,klimaticheskaya_tehnika,otopitelnye_pechi,11,0,NaN,другой,бюджет,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3235,https://hi-tech.md/bytovaya-tehnika/klimatiche...,T-900000006,2024-06-09,Предзаказ,Печь отопительно-варочная Огонь-батарея 9 антр...,6289.0,NaN,0,NaN,bytovaya_tehnika,klimaticheskaya_tehnika,otopitelnye_pechi,11,0,NaN,другой,бюджет,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3297,https://hi-tech.md/bytovaya-tehnika/klimatiche...,T-900000002,2024-06-09,Предзаказ,Печь отопительно-варочная Огонь-батарея 11 ант...,6899.0,NaN,0,NaN,bytovaya_tehnika,klimaticheskaya_tehnika,otopitelnye_pechi,11,0,NaN,другой,бюджет,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3300,https://hi-tech.md/bytovaya-tehnika/klimatiche...,T-900000001,2024-06-09,Предзаказ,Печь отопительно-варочная Огонь-батарея 11 ант...,6939.0,NaN,0,NaN,bytovaya_tehnika,klimaticheskaya_tehnika,otopitelnye_pechi,11,0,NaN,другой,бюджет,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3301,https://hi-tech.md/bytovaya-tehnika/klimatiche...,T-900000007,2024-06-09,Предзаказ,Печь отопительная Валериан 15 кВт антрацит EU,6973.0,NaN,0,NaN,bytovaya_tehnika,klimaticheskaya_tehnika,otopitelnye_pechi,11,0,NaN,квт,бюджет,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3412,https://hi-tech.md/bytovaya-tehnika/klimatiche...,T-900000008,2024-06-09,Предзаказ,Печь отопительная Валериан 20 кВт антрацит EU,8059.0,NaN,0,NaN,bytovaya_tehnika,klimaticheskaya_tehnika,otopitelnye_pechi,11,0,NaN,квт,бюджет,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3624,https://hi-tech.md/bytovaya-tehnika/klimatiche...,T-900000011,2024-06-09,Предзаказ,Печь-камин отопительно-варочная Статика Секста...,11530.0,NaN,0,NaN,bytovaya_tehnika,klimaticheskaya_tehnika,otopitelnye_pechi,11,0,NaN,другой,стандарт,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [64]:
df.drop(['count_sity'], axis= 1 , inplace= True )

In [65]:
preprocessing(df)

таблица имеет следующий вид:


,soup_name,id,date,availability,title,price,old_price,action_sale,sale,category,subcategory,group,count_group,count_group_presence,dif_price,brend,group_prise,Тир_Юности_14А,Ц_Тир_25_Октября_69,ТИРАСПОЛЬ,Бенд_Спортивная_5,Бенд_Советская_5,Бенд_Советская_8,БЕНДЕРЫ,СЛОБОДЗЕЯ,ДНЕСТРОВСК,Первомайск,Григ_КМаркса_181А,Григ_КМаркса_172,Григориополь,ДУБОССАРЫ,Рыбн_Вальченко_43,Рыбн_Кирова_135,РЫБНИЦА,Каменка,error
0,https://hi-tech.md/televizory-i-elektronika/ba...,Т-000088228,2024-06-09,В наличии,Батарейка Energenie EG-BA-CR1220-01,4.0,NaN,0,NaN,televizory_i_elektronika,batareyki_i_zaryadnye_ustroystva,batareyki_i_zaryadnye_ustroystva,76,76,199.75,energenie,бюджет,1,1,2,1,1,0,2,1,1,1,1,1,2,1,1,1,2,1,0
1,https://hi-tech.md/televizory-i-elektronika/ba...,Т-000088229,2024-06-09,В наличии,Батарейка Energenie EG-BA-CR2016-01,4.0,NaN,0,NaN,televizory_i_elektronika,batareyki_i_zaryadnye_ustroystva,batareyki_i_zaryadnye_ustroystva,76,76,199.75,energenie,бюджет,1,1,2,1,1,0,2,1,1,1,1,1,2,1,1,1,2,1,0
2,https://hi-tech.md/televizory-i-elektronika/ba...,Т-000088221,2024-06-09,В наличии,Батарейка Energenie EG-BA-AA4-01,5.0,NaN,0,NaN,televizory_i_elektronika,batareyki_i_zaryadnye_ustroystva,batareyki_i_zaryadnye_ustroystva,76,76,199.75,energenie,бюджет,1,1,2,1,1,1,3,1,1,1,1,1,2,1,1,1,2,1,0
3,https://hi-tech.md/televizory-i-elektronika/ba...,Т-000088222,2024-06-09,В наличии,Батарейка Energenie EG-BA-AAA4-01,5.0,NaN,0,NaN,televizory_i_elektronika,batareyki_i_zaryadnye_ustroystva,batareyki_i_zaryadnye_ustroystva,76,76,199.75,energenie,бюджет,1,1,2,1,1,1,3,1,1,1,1,1,2,1,1,1,2,1,0
4,https://hi-tech.md/televizory-i-elektronika/ba...,Т-000088230,2024-06-09,В наличии,Батарейка Energenie EG-BA-CR2025-01,5.0,NaN,0,NaN,televizory_i_elektronika,batareyki_i_zaryadnye_ustroystva,batareyki_i_zaryadnye_ustroystva,76,76,199.75,energenie,бюджет,1,1,2,1,1,0,2,1,1,1,1,1,2,1,1,1,2,1,0


****************************************************************************************************
таблица имеет 8722 сторок и 36 столбцов
****************************************************************************************************
в таблице обнаружено дублекатов:0
****************************************************************************************************
в таблице обнаружены пропуски в следующих столбцах:


soup_name                  0
id                         0
date                       0
availability               0
title                      0
price                      0
old_price               6584
action_sale                0
sale                    6584
category                   0
subcategory                0
group                      0
count_group                0
count_group_presence       0
dif_price                 26
brend                      0
group_prise                0
Тир_Юности_14А             0
Ц_Тир_25_Октября_69        0
ТИРАСПОЛЬ                  0
Бенд_Спортивная_5          0
Бенд_Советская_5           0
Бенд_Советская_8           0
БЕНДЕРЫ                    0
СЛОБОДЗЕЯ                  0
ДНЕСТРОВСК                 0
Первомайск                 0
Григ_КМаркса_181А          0
Григ_КМаркса_172           0
Григориополь               0
ДУБОССАРЫ                  0
Рыбн_Вальченко_43          0
Рыбн_Кирова_135            0
РЫБНИЦА                    0
Каменка       

****************************************************************************************************
Эти пропуски составлябт следующее количество в процентах


soup_name                0.0
id                       0.0
date                     0.0
availability             0.0
title                    0.0
price                    0.0
old_price               75.5
action_sale              0.0
sale                    75.5
category                 0.0
subcategory              0.0
group                    0.0
count_group              0.0
count_group_presence     0.0
dif_price                0.3
brend                    0.0
group_prise              0.0
Тир_Юности_14А           0.0
Ц_Тир_25_Октября_69      0.0
ТИРАСПОЛЬ                0.0
Бенд_Спортивная_5        0.0
Бенд_Советская_5         0.0
Бенд_Советская_8         0.0
БЕНДЕРЫ                  0.0
СЛОБОДЗЕЯ                0.0
ДНЕСТРОВСК               0.0
Первомайск               0.0
Григ_КМаркса_181А        0.0
Григ_КМаркса_172         0.0
Григориополь             0.0
ДУБОССАРЫ                0.0
Рыбн_Вальченко_43        0.0
Рыбн_Кирова_135          0.0
РЫБНИЦА                  0.0
Каменка       

****************************************************************************************************
столбцы имеют следующие типы:


soup_name                       object
id                              object
date                    datetime64[ns]
availability                    object
title                           object
price                          float64
old_price                      float64
action_sale                      int64
sale                           float64
category                        object
subcategory                     object
group                           object
count_group                      int64
count_group_presence             int64
dif_price                      float64
brend                           object
group_prise                   category
Тир_Юности_14А                   int32
Ц_Тир_25_Октября_69              int32
ТИРАСПОЛЬ                        int32
Бенд_Спортивная_5                int32
Бенд_Советская_5                 int32
Бенд_Советская_8                 int32
БЕНДЕРЫ                          int32
СЛОБОДЗЕЯ                        int32
ДНЕСТРОВСК               

****************************************************************************************************
названия столбцов переписаны в нижнем регистре, пробелы заменены на нижнее подчеркивание
****************************************************************************************************
<class 'pandas.core.frame.DataFrame'>
Int64Index: 8722 entries, 0 to 9292
Data columns (total 36 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   soup_name             8722 non-null   object        
 1   id                    8722 non-null   object        
 2   date                  8722 non-null   datetime64[ns]
 3   availability          8722 non-null   object        
 4   title                 8722 non-null   object        
 5   price                 8722 non-null   float64       
 6   old_price             2138 non-null   float64       
 7   action_sale           8722 non-null   int64         
 8   sale             

None

In [66]:
df.to_csv('csv/hi_tech_end_'+str(today.year)+'_'+str(today.month)+'_'+str(today.day)+'.csv', index=False)  

In [67]:
df.loc[3183,'id']

'Т-000071796'

In [68]:
bed = 'T-900000003'